<a href="https://colab.research.google.com/github/rg-smith/remote-sensing-hydro/blob/main/labs/lab2/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2

In this lab, you will use python (and a module called Google Earth Engine) to view satellite imagery, and access precipitation datasets.

Before starting the sections, we will import the modules we need, initialize Google Earth Engine, and define some functions.

The first block of code only needs to be run once, the first time you open your session (if you close out of the session and open again, you will need to run this again). Follow the prompts to initialize earth engine. You will be taken to a link where you need to give permission to link your google account with Google Earth Engine, then copy and paste some text below the code block.

In [ ]:
import ee
import folium
import numpy as np
import branca.colormap as cm

from google.colab import drive
drive.mount('/content/drive/')

ee.Authenticate()
ee.Initialize()

This block of code also only needs to be run once. It is defining a bunch of functions that you will use in this exercise. It is not important for you to understand the code here for this exercise.

In [7]:
# functions needed for this lab (and some other useful ones that you can use if you're interested)

# to add a layer to our map:
def add_ee_layer(self, ee_object, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):  
            range = ee.Image(ee_object).reduceRegion(ee.Reducer.percentile([1, 99]),scale=10000)
            vals = range.getInfo()
            min=list(vals.items())[0][1]
            max=list(vals.items())[1][1]
            vis = {'min': min, 'max': max, 'palette': ['0000FF', 'FFFFFF','FF0000']}

            map_id_dict = ee.Image(ee_object).getMapId(vis)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
            colormap = cm.LinearColormap(vmin=min,vmax=max,colors=['blue', 'white','red']).to_step(n=10)
            colormap.caption=name
            self.add_child(colormap)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except Exception as e:
        print("Could not display {}".format(name))
        print(e)

# to convert a google earth engine image to a python array
def to_array(img,aoi):
  band_arrs = img.sampleRectangle(region=aoi,properties=['scale=1000'],defaultValue=-999)

  band_names=img.bandNames().getInfo()

  for kk in range(len(band_names)):
    if kk==0:
      dat1=np.array(band_arrs.get(band_names[kk]).getInfo())
      dat_full=np.zeros((dat1.shape[0],dat1.shape[1],len(band_names)))
      dat_full[:,:,kk]=dat1
    else:
      dat=np.array(band_arrs.get(band_names[kk]).getInfo())
      dat_full[:,:,kk]=dat
  return(dat_full)

# to calculate an index
def getIndex(image,b1,b2):
  return image.normalizedDifference([b1, b2])

# to calculate a ratio
def getRatio(image1,image2):
  ratio=image1.divide(image2)
  return ratio

# to create a color map from a specific image
def getVisparams(image,aoi):
  range = image.reduceRegion(ee.Reducer.percentile([1, 99]),aoi,300)
  vals = range.getInfo()
  min=list(vals.items())[0][1]
  max=list(vals.items())[1][1]
  visParams = {'min': min, 'max': max, 'palette': ['0000FF', 'FFFFFF','FF0000']}
  return(visParams)

# to get the link to download an earth engine image
def getLink(image,aoi):
  link = image.getDownloadURL({
    'scale': 1000,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': aoi})
  print(link)

# create an earth engine geometry polygon
def addGeometry(min_lon,max_lon,min_lat,max_lat):
  import ee
  geom = ee.Geometry.Polygon(
      [[[min_lon, max_lat],
        [min_lon, min_lat],
        [max_lon, min_lat],
        [max_lon, max_lat]]])
  return(geom)

# load prism data
def get_prism_image(date1,date2,geometry):
  import ee
  prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81m')
  prism_img = prism.filterDate(date1,date2).select('ppt').mean().clip(geometry)
  return(prism_img) # returns prism average monthly precipitation, in mm

# load landsat 8 data
def get_l8_image(date1,date2,geometry):
  import ee
  l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT')
  l8_img = l8.filterDate(date1,date2).mean().clip(geometry)
  return(l8_img)

def export_to_drive(raster,filename,foldername,geometry):
  # Export the image, specifying scale and region.
  task = ee.batch.Export.image.toDrive(**{
      'image': raster,
      'description': filename,
      'folder': foldername,
      'fileNamePrefix': filename,
      'scale': 1000,
      'region': geometry,
      'fileFormat': 'GeoTIFF',
      'formatOptions': {
        'cloudOptimized': 'true'
      },
  })
  task.start()

def get_elev(geometry):
  import ee
  elev = ee.Image('USGS/NED').clip(geometry)
  return(elev)

def get_gpm_image(date1,date2,geometry):
  import ee
  gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_MONTHLY_V06')
  gpm_img = gpm.filterDate(date1,date2).select('precipitation').mean().multiply(24*365/12).clip(geometry)
  return(gpm_img) # returns gpm average monthly precipitation in mm

# load sentinel 2 data
def get_s2_image(date1,date2,geometry):
    import ee
    s2 = ee.ImageCollection('COPERNICUS/S2')
    s2_img = s2.filterDate(date1,date2).filterBounds(geometry).first().clip(geometry)
    return(s2_img)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Part 1: Viewing precipitation data for a given geometry and date

 

First, we will define the study area and time period and load some Landsat 8 data

In [4]:
# create a bounding box that defines the study area
geom = addGeometry(-110, -105,40,45) # min long, max long, min lat, max lat

# define dates of interest (inclusive).
start = '2020-04-01'
end = '2021-04-01' #can go up to april 2021

# get elevation data
elev = get_elev(geom)

# now get gpm precipitation over the same region for a specified time period
gpm_img = get_gpm_image(start,end,geom)

# now get prism precipitation over the same time period/region
prism_img = get_prism_image(start,end,geom)

Now, we will view the map. Scroll to the top so you can check/uncheck layers. Observe areas that are likely to have Gossans. Take a screenshot to include in your lab report, and describe the wavelengths you used for your ratios.

In [ ]:
my_map = folium.Map(location=[42.5, -107], zoom_start=6)

# Add the land cover to the map object.
#my_map.add_ee_layer(l8_img,'Landsat 8')
my_map.add_ee_layer(elev,'Elevation')
my_map.add_ee_layer(prism_img,'PRISM precip')
my_map.add_ee_layer(gpm_img,'GPM precip')

my_map.add_ee_layer(geom,'bounding box')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

Now, we will export the tiffs to your Google Colab folder (this is a folder in your Google Drive storage). We will then use some other python functions to do analysis. After you run the code below, it will take a few minutes to save the files to your Google Drive. Move on to the next step while you're waiting.

In [11]:
export_to_drive(gpm_img,'lab2_gpm_precip_CO','Colab Notebooks',geom) # raster, file name, folder name, geometry to clip it with
export_to_drive(elev,'lab2_elev_CO','Colab Notebooks',geom)
export_to_drive(prism_img,'lab2_prism_precip_CO','Colab Notebooks',geom)

While you're waiting, try a different way of downloading the rasters below. Once you have downloaded them, load them into QGIS. Reproject the elevation dataset to a projected coordinate reference system (UTM Zone 15N, WGS84 is a good choice). You can do this by right-clicking on the elevation file, Export, then specify the coordinate reference system.

Calculate the slope of the projected coordinate reference system (Raster -> Analysis -> Slope).

Identify an area with high landslide risk (high slope/high precipitation), and make a shapefile  over that region.

Take a screenshot and include with your report.

The precipitation units are the average monthly precipitation in mm. What is the maximum/minimum total precipitation over the study period?

In [ ]:
print('GPM precipitation raster download:')
getLink(gpm_img,geom)
print('Elevation raster download:')
getLink(elev,geom)
print('PRISM precipitation raster download:')
getLink(prism_img,geom)

Now, we will load the rasters we just downloaded with a separate python package--rasterio--and compare GPM with PRISM.

In [ ]:
!pip install rasterio
import rasterio
from rasterio.warp import reproject
import matplotlib.pyplot as plt

Click on the 'Files' tab on the left. Go to drive > MyDrive > Colab Notebooks. You should see some '.tif' files for prism, gpm and elevation data. If you don't see them yet, you may have to wait a few minutes for them to get saved to the cloud.

Now you will load the rasters into python with rasterio. This package works more easily with the usual python packages, making plotting easier.

In [24]:
folder_location='drive/MyDrive/Colab Notebooks/'
gpm=rasterio.open(folder_location+'lab2_gpm_precip_CO.tif')
prism=rasterio.open(folder_location+'lab2_prism_precip_CO.tif')
elev=rasterio.open(folder_location+'lab2_elev_CO.tif')

Now, we will make a crossplot of PRISM and GPM data, with points colored by elevation. PRISM are considered more accurate generally. Comment on the accuracy of the GPM data.

In [ ]:
gpm_flat=gpm.read().flatten()
prism_flat=prism.read().flatten()
elev_flat=elev.read().flatten()
plt.figure(figsize=(12,8));plt.scatter(gpm_flat,prism_flat,c=elev_flat,s=1)
reference_line=[np.nanmin(gpm_flat),np.nanmax(gpm_flat)]
plt.plot(reference_line,reference_line,'r')
plt.xlabel('GPM precipitation, mm/month')
plt.ylabel('PRISM precipitation, mm/month')
plt.colorbar()

# Part 2: Repeat the above process, but this time do it over the study area for your term project

If you haven't decided on a location for your project yet, you can select an area you are considering for your term project. It's OK to change later.

You will need to 'fill in the blanks' for the code below.

In [ ]:
# create a bounding box that defines the study area. This will cover the Lake of the Ozarks region
geom_study_area = addGeometry(?,?,?,?) # min long, max long, min lat, max lat

# define dates of interest (inclusive).
start = '?'
end = '?'

# use the tools in the example from Part 1 to pull GPM precip and elevation data

Now, you will display the map. Try at least three different indices. Include a screenshot and description of what wavelengths you used for each one in your lab report.

In [ ]:
my_map = folium.Map(location=[?, ?], zoom_start=11) # location values should be lat, lon. zoom_start: lower values are more zoomed out. Adjust until it looks right.

# Add the land cover to the map object.
my_map.add_ee_layer(?,'? label')
my_map.add_ee_layer(?,'? label')
my_map.add_ee_layer(geom_study_area,'bounding box')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

Download these rasters and display in QGIS as you did before. Repeat the same exercise -- reproject the elevation to a projected CRS, then calculate slope and identify an area of high landslide risk.

As before, determine the average precipitation over the study period.

In [ ]:
print('GPM precipitation raster download:')
getLink(?,geom_study_area)
print('Elevation raster download:')
getLink(?,geom_study_area)
print('PRISM precipitation raster download')
getLink(?,geom)

If you'd like to get more familiar with python, try creating a new code block to do the same analysis as before (load the rasters with rasterio, then make a crossplot). It's mostly just copying code from above, and changing filenames. This part is not required, but if you try it, you will get more comfortable with python.